In [1]:
import pandas as pd
import numpy as np
import json

with open("/Users/yagel/Documents/nidap_temp/postals_NL.json", "r") as f:
    postals = json.load(f)


In [2]:
#
indeling = pd.read_excel('/Users/yagel/Downloads/IndelingOpleidingen.xlsx')

In [3]:
#
hbo = pd.read_excel('/Users/yagel/Library/CloudStorage/OneDrive-SharedLibraries-NIDAP/NIDAP-Project - Documents/Datasets-Nidap analyses/Brondata/DUO/Ho instroom merge/03b.-eerstejaars-ingeschrevenen-hbo-domein-hbo-2014-2024.xlsx', sheet_name='Sheet1')
wo = pd.read_excel('/Users/yagel/Library/CloudStorage/OneDrive-SharedLibraries-NIDAP/NIDAP-Project - Documents/Datasets-Nidap analyses/Brondata/DUO/Ho instroom merge/03b.-eerstejaars-ingeschrevenen-wo-domein-wo-2014-2024.xlsx')

#
hbo.loc[:, 'Niveau'] = 'hbo'
wo.loc[:, 'Niveau'] = 'wo'

#
opleidingen = pd.concat((hbo, wo))

#
if 'Verwant' in opleidingen.columns:
    opleidingen = opleidingen.drop('Verwant', axis=1)
    

In [4]:
# Select columns for the years and non-numeric columns
year_columns = [str(year) for year in range(2014, 2025)]
categorical_columns = [col for col in opleidingen.columns if col not in year_columns + ['GESLACHT']]

# Group by 'GESLACHT' and other categorical columns, and sum the year columns
opleidingen = opleidingen.groupby(categorical_columns)[year_columns].sum().reset_index().copy()

In [5]:
#
opleidingen = opleidingen.pivot_table(
    index=['INSTELLINGSNAAM ACTUEEL', 'OPLEIDINGSCODE ACTUEEL', 'OPLEIDINGSNAAM ACTUEEL', 'OPLEIDINGSVORM', 'TYPE HOGER ONDERWIJS', 'GEMEENTENAAM', 'Niveau'],
    values=year_columns,
).reset_index().copy()

In [6]:
#
naam_sub_nidap = indeling[['Opleidingsnaam', 'Subcategorie - NIDAP']].drop_duplicates().set_index('Opleidingsnaam').T.to_dict(orient='index')['Subcategorie - NIDAP']
code_sub_nidap = indeling[['Opleidingscode', 'Subcategorie - NIDAP']].drop_duplicates().set_index('Opleidingscode').T.to_dict(orient='index')['Subcategorie - NIDAP']
naam_hoofd_nidap = indeling[['Opleidingsnaam', 'Hoofdcategorie - NIDAP']].drop_duplicates().set_index('Opleidingsnaam').T.to_dict(orient='index')['Hoofdcategorie - NIDAP']
code_hoofd_nidap = indeling[['Opleidingscode', 'Hoofdcategorie - NIDAP']].drop_duplicates().set_index('Opleidingscode').T.to_dict(orient='index')['Hoofdcategorie - NIDAP']

#
naam_sub_isced = indeling[['Opleidingsnaam', 'ISCED-subgroep']].drop_duplicates().set_index('Opleidingsnaam').T.to_dict(orient='index')['ISCED-subgroep']
code_sub_isced = indeling[['Opleidingscode', 'ISCED-subgroep']].drop_duplicates().set_index('Opleidingscode').T.to_dict(orient='index')['ISCED-subgroep']
naam_hoofd_isced = indeling[['Opleidingsnaam', 'ISCED-hoofdgroep']].drop_duplicates().set_index('Opleidingsnaam').T.to_dict(orient='index')['ISCED-hoofdgroep']
code_hoofd_isced = indeling[['Opleidingscode', 'ISCED-hoofdgroep']].drop_duplicates().set_index('Opleidingscode').T.to_dict(orient='index')['ISCED-hoofdgroep']

#
naam_sub_isced = indeling[['Opleidingsnaam', 'ROA Opleidingstype']].drop_duplicates().set_index('Opleidingsnaam').T.to_dict(orient='index')['ROA Opleidingstype']
code_sub_isced = indeling[['Opleidingscode', 'ROA Opleidingstype']].drop_duplicates().set_index('Opleidingscode').T.to_dict(orient='index')['ROA Opleidingstype']

In [7]:
def match_indeling(val, dict):
    #
    try:
        return dict[val]
    except KeyError:
        return pd.NA

In [8]:
#
opleidingen.loc[:, 'Subcategorie - NIDAP'] = opleidingen['OPLEIDINGSNAAM ACTUEEL'].apply(lambda x : match_indeling(x, naam_sub_nidap))
opleidingen.loc[opleidingen['Subcategorie - NIDAP'].isna(), 'Subcategorie - NIDAP'] = opleidingen.loc[opleidingen['Subcategorie - NIDAP'].isna(),'OPLEIDINGSCODE ACTUEEL'].apply(lambda x : match_indeling(x, code_sub_nidap))

#
opleidingen.loc[:, 'Hoofdcategorie - NIDAP'] = opleidingen['OPLEIDINGSNAAM ACTUEEL'].apply(lambda x : match_indeling(x, naam_hoofd_nidap))
opleidingen.loc[opleidingen['Hoofdcategorie - NIDAP'].isna(), 'Hoofdcategorie - NIDAP'] = opleidingen.loc[opleidingen['Hoofdcategorie - NIDAP'].isna(),'OPLEIDINGSCODE ACTUEEL'].apply(lambda x : match_indeling(x, code_hoofd_nidap))

#
opleidingen.loc[:, 'ISCED-subgroep'] = opleidingen['OPLEIDINGSNAAM ACTUEEL'].apply(lambda x : match_indeling(x, naam_sub_isced))
opleidingen.loc[opleidingen['ISCED-subgroep'].isna(), 'ISCED-subgroep'] = opleidingen.loc[opleidingen['ISCED-subgroep'].isna(),'OPLEIDINGSCODE ACTUEEL'].apply(lambda x : match_indeling(x, code_sub_isced))

#
opleidingen.loc[:, 'ISCED-hoofdgroep'] = opleidingen['OPLEIDINGSNAAM ACTUEEL'].apply(lambda x : match_indeling(x, naam_hoofd_isced))
opleidingen.loc[opleidingen['ISCED-hoofdgroep'].isna(), 'ISCED-hoofdgroep'] = opleidingen.loc[opleidingen['ISCED-hoofdgroep'].isna(),'OPLEIDINGSCODE ACTUEEL'].apply(lambda x : match_indeling(x, code_hoofd_isced))

#
opleidingen.loc[:, 'ROA Opleidingstype'] = opleidingen['OPLEIDINGSNAAM ACTUEEL'].apply(lambda x : match_indeling(x, naam_sub_isced))
opleidingen.loc[opleidingen['ROA Opleidingstype'].isna(), 'ROA Opleidingstype'] = opleidingen.loc[opleidingen['ROA Opleidingstype'].isna(),'OPLEIDINGSCODE ACTUEEL'].apply(lambda x : match_indeling(x, code_sub_isced))

In [9]:
#
query_gemeente_provincie = pd.DataFrame(postals).T[['Gemeente name', 'Provincie name']].drop_duplicates().set_index('Gemeente name').T.to_dict(orient='index')['Provincie name']

In [10]:
# convert all postal codes to gemeente
opleidingen['Provincie'] = opleidingen['GEMEENTENAAM'].apply(lambda x : match_indeling(x, query_gemeente_provincie))

In [11]:
#
arb_reg = pd.read_excel('/Users/yagel/Library/CloudStorage/OneDrive-SharedLibraries-NIDAP/NIDAP-Project - Documents/Datasets-Nidap analyses/Brondata/Arbeidsmarktregios/Arbeidsmarktregios (08-07-2025).xlsx')

#
arb_reg = arb_reg.to_dict(orient='list')

inverted_dict = {}

for key, l in arb_reg.items():
    for value in l:
        inverted_dict[str(value).lower().replace('’', "'")] = key

#
opleidingen.loc[:, 'Arbeidsmarktregio'] = opleidingen.GEMEENTENAAM.str.lower().apply(lambda x : match_indeling(x, inverted_dict))

In [12]:
#
opleidingen['Gem. 2023 - 2024'] = (opleidingen['2023'] + opleidingen['2024']) / 2
opleidingen['Trend 2018 - 2024'] = (opleidingen['2023'] + opleidingen['2024']) / (opleidingen['2018'] - opleidingen['2019']) - 1

#
opleidingen['Trend 2018 - 2024'] = opleidingen['Trend 2018 - 2024'].replace(np.inf, pd.NA)

In [13]:
#
opleidingen = opleidingen.drop(year_columns, axis=1).copy()

In [14]:
#
opleidingen.to_json('resultaten_gemeente.json', index=False, orient="records")

In [15]:
#
opleidingen = pd.read_json('resultaten_gemeente.json')#, index=False, orient="records")

In [22]:
opleidingen.columns

Index(['INSTELLINGSNAAM ACTUEEL', 'OPLEIDINGSCODE ACTUEEL',
       'OPLEIDINGSNAAM ACTUEEL', 'OPLEIDINGSVORM', 'TYPE HOGER ONDERWIJS',
       'GEMEENTENAAM', 'Niveau', 'Subcategorie - NIDAP',
       'Hoofdcategorie - NIDAP', 'ISCED-subgroep', 'ISCED-hoofdgroep',
       'Provincie', 'Arbeidsmarktregio', 'Gem. 2023 - 2024',
       'Trend 2018 - 2024'],
      dtype='object')

In [23]:
opleidingen['TYPE HOGER ONDERWIJS'].unique()

array(['bachelor', 'master', 'associate degree', 'ongedeelde opleiding',
       'postmaster'], dtype=object)

In [ ]:
# - Meer uitleg bij de visualisatie
# - meerdere categoriën
# - zelf opleidingen selecteren
# - standaard op WO

In [117]:
opleidingen = opleidingen.loc[opleidingen["Gem. 2023 - 2024"] > 0].copy()

In [120]:
opleidingen.loc[opleidingen['Niveau'].isin(['wo']) * opleidingen['Subcategorie - NIDAP'].isin(['Data Science']), 'INSTELLINGSNAAM ACTUEEL'].value_counts()

INSTELLINGSNAAM ACTUEEL
Technische Universiteit Eindhoven      2
Tilburg University                     2
Erasmus Universiteit Rotterdam         1
Rijksuniversiteit Groningen            1
Universiteit Leiden                    1
Universiteit Utrecht                   1
Universiteit van Amsterdam             1
Wageningen University                  1
transnationale Universiteit Limburg    1
Name: count, dtype: int64